In [2]:
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import matplotlib.pyplot as plt
%matplotlib inline

/home/batman/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [5]:
# Reading the saved data pickle file
df_stocks = pd.read_csv('DataFina/beforeClean.csv')
# df_stocks = df_stocks.iloc[:,1:]

df_stocks = df_stocks.iloc[:,[1,8,9]]
df_stocks

,Date,Adj Close,news
0,2010-03-17,0.000000,cerc can cap trading margins: sc # pranab warn...
1,2010-03-18,0.007942,hc declines to stay 3g auction plan # 'to be o...
2,2010-03-19,0.024598,green hurdle: alternate mines for power plants...
3,2010-03-22,-0.012212,bharti ties up $8.3bn for zain # shoprite buy ...
4,2010-03-23,0.021253,indian it poised to gain from us health reform...
5,2010-03-25,0.033071,sail selloff may come before cabinet today # '...
6,2010-03-26,-0.000848,sbi may extend teaser home loan # people integ...
7,2010-03-29,0.001647,govt to monitor ipo proceeds # satyam fraud hi...
8,2010-03-30,0.004135,japan promises over rs 10k cr in infra project...
9,2010-03-31,0.005011,your savings bank account will now earn 16-18%...


In [6]:
df_stocks = df_stocks.dropna(axis=0, how='any')
# df_stocks = df_stocks.iloc[:,[1,5,6,8,9]]
df_stocks.shape

(1925, 3)

In [8]:
# df_stocks['Adj Close'] = df_stocks['Adj Close']*100
def modify_data(s):
    
    if float(s) < 0:
        s = 0
    elif float(s) == 0:
        s = 1
    elif float(s) >0:
        s = 1
    return s
i = 0
size = df_stocks.shape[0]
while i<size:
    df_stocks.iloc[i,1]= modify_data(df_stocks.iloc[i,1])
    i = i+1

In [5]:
# selecting the prices and articles
# df_stocks = df_stocks[['Title', 'Adj Close']]

In [9]:
df_stocks.loc[:,'news'] = df_stocks.loc[:,'news'].map(lambda x: x.lstrip('.-'))

In [10]:
df_stocks

,Date,Adj Close,news
0,2010-03-17,1.0,cerc can cap trading margins: sc # pranab warn...
1,2010-03-18,1.0,hc declines to stay 3g auction plan # 'to be o...
2,2010-03-19,1.0,green hurdle: alternate mines for power plants...
3,2010-03-22,0.0,bharti ties up $8.3bn for zain # shoprite buy ...
4,2010-03-23,1.0,indian it poised to gain from us health reform...
5,2010-03-25,1.0,sail selloff may come before cabinet today # '...
6,2010-03-26,0.0,sbi may extend teaser home loan # people integ...
7,2010-03-29,1.0,govt to monitor ipo proceeds # satyam fraud hi...
8,2010-03-30,1.0,japan promises over rs 10k cr in infra project...
9,2010-03-31,1.0,your savings bank account will now earn 16-18%...


In [11]:
df = df_stocks[['Adj Close']].copy()
df['Date'] = df_stocks['Date']

In [12]:
# Adding new columns to the data frame
df["compound"] = ''
df["neg"] = ''
df["neu"] = ''
df["pos"] = ''

In [13]:
df

,Adj Close,Date,compound,neg,neu,pos
0,1.0,2010-03-17,,,,
1,1.0,2010-03-18,,,,
2,1.0,2010-03-19,,,,
3,0.0,2010-03-22,,,,
4,1.0,2010-03-23,,,,
5,1.0,2010-03-25,,,,
6,0.0,2010-03-26,,,,
7,1.0,2010-03-29,,,,
8,1.0,2010-03-30,,,,
9,1.0,2010-03-31,,,,


In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
sid = SentimentIntensityAnalyzer()
for date, row in df_stocks.T.iteritems():
    try:
        sentence = unicodedata.normalize('NFKD', df_stocks.loc[date, 'news'])#.encode('ascii','ignore')
        ss = sid.polarity_scores(sentence)
        df.set_value(date, 'compound', ss['compound'])
        df.set_value(date, 'neg', ss['neg'])
        df.set_value(date, 'neu', ss['neu'])
        df.set_value(date, 'pos', ss['pos'])
    except TypeError:
        print( df_stocks.loc[date, 'news'])
        print (date)
# dtest = xgb.DMatrix(np.array(x_test))

In [15]:
df.Date = df.Date

In [16]:
# train_start_date = '2013-12-30'
# train_end_date = '2017-03-17'
# test_start_date = '2017-03-18'
# test_end_date = '2018-3-16'

train = df[df['Date'] < '2017-03-17']
test = df[df['Date'] > '2017-03-18']

# train = df.iloc[train_start_date : train_end_date]
# test = df.ix[test_start_date:test_end_date]
# train.to_csv('Data/train_pnb.csv',index=None)
# test.to_csv('Data/test_pnb.csv',index=None)
test
x_test = test.iloc[:,2:]
y_test = test.iloc[:,0]

# X
test

,Adj Close,Date,compound,neg,neu,pos
1728,1.0,2017-03-20,-0.9887,0.131,0.752,0.118
1729,0.0,2017-03-21,-0.9971,0.136,0.768,0.096
1730,0.0,2017-03-22,0.9982,0.057,0.853,0.09
1731,0.0,2017-03-23,0.9843,0.086,0.817,0.097
1732,1.0,2017-03-24,-0.9977,0.122,0.783,0.095
1733,1.0,2017-03-27,-0.9996,0.153,0.764,0.083
1734,1.0,2017-03-28,-0.991,0.125,0.777,0.098
1735,1.0,2017-03-29,-0.9981,0.113,0.807,0.081
1736,1.0,2017-03-30,0.9921,0.089,0.812,0.099
1737,0.0,2017-03-31,0.9995,0.072,0.805,0.124


In [17]:

print(train.shape)
train = train.dropna(axis=0, how='any')
print(train.shape)
Y = train['Adj Close']
train_new = train.drop(['Adj Close'],axis = 1)
X=train_new
X = X.iloc[:,1:]
# test_1 = test_1.drop('nLastTradedPrice',axis=1)
# X
# X.to_csv('traintatti.csv')
# Y.to_csv('traintarget.csv')
X

(1727, 6)
(1727, 6)


,compound,neg,neu,pos
0,-0.996,0.125,0.799,0.077
1,-0.9992,0.171,0.729,0.101
2,-0.9944,0.11,0.804,0.086
3,-0.9501,0.115,0.782,0.103
4,-0.9967,0.111,0.821,0.068
5,0.6732,0.084,0.833,0.082
6,-0.9932,0.145,0.732,0.123
7,-0.9913,0.121,0.772,0.107
8,-0.9315,0.097,0.8,0.103
9,-0.9982,0.136,0.783,0.081


In [18]:
import xgboost as xgb

# Y = np.array(Y,dtype=float)
# Y = pd.DataFrame(Y)
# Y.to_csv('tatti.csv')
# Y = Y.dropna(axis=0, how='any')
# train.shape
# X.to_csv('XG_in.csv',index=None)
# Y
dtrain = xgb.DMatrix(np.array(X), label=Y)
# i = 0
# row = X.shape[0]
# while i<row:
#     X.iloc[i,0] = float(X.iloc[i,0])
#     X.iloc[i,1]= float(X.iloc[i,1])
#     X.iloc[i,2]= float(X.iloc[i,2])
#     X.iloc[i,3]= float(X.iloc[i,3])
#     Y.iloc[i] = float(Y.iloc[i])
#     i = i + 1
#     print(i)


/home/batman/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
print(X)
x_test

     compound    neg    neu    pos
0      -0.996  0.125  0.799  0.077
1     -0.9992  0.171  0.729  0.101
2     -0.9944   0.11  0.804  0.086
3     -0.9501  0.115  0.782  0.103
4     -0.9967  0.111  0.821  0.068
5      0.6732  0.084  0.833  0.082
6     -0.9932  0.145  0.732  0.123
7     -0.9913  0.121  0.772  0.107
8     -0.9315  0.097    0.8  0.103
9     -0.9982  0.136  0.783  0.081
10     -0.946  0.111  0.781  0.108
11    -0.9938  0.118  0.794  0.089
12    -0.9963   0.12  0.788  0.092
13    -0.9989  0.134  0.801  0.065
14    -0.9982  0.139  0.783  0.077
15     -0.995  0.119  0.791  0.091
16    -0.9095  0.085  0.839  0.077
17    -0.9972  0.154  0.728  0.118
18    -0.9982  0.142  0.776  0.082
19    -0.9982  0.132  0.769  0.099
20    -0.9979  0.141  0.785  0.075
21     0.9709  0.089  0.808  0.104
22     0.9166  0.087  0.815  0.098
23    -0.9909  0.098  0.829  0.072
24     0.9179  0.109  0.772  0.119
25    -0.9696  0.091  0.841  0.068
26    -0.9899  0.125   0.79  0.085
27    -0.9833  0.085

,compound,neg,neu,pos
1728,-0.9887,0.131,0.752,0.118
1729,-0.9971,0.136,0.768,0.096
1730,0.9982,0.057,0.853,0.09
1731,0.9843,0.086,0.817,0.097
1732,-0.9977,0.122,0.783,0.095
1733,-0.9996,0.153,0.764,0.083
1734,-0.991,0.125,0.777,0.098
1735,-0.9981,0.113,0.807,0.081
1736,0.9921,0.089,0.812,0.099
1737,0.9995,0.072,0.805,0.124


In [20]:

import xgboost as xgb
import operator

dtrain = xgb.DMatrix(np.array(X), label=Y)
#dval=xgb.DMatrix(np.array(x_val),label=y_val)
dtest = xgb.DMatrix(np.array(x_test))
xgb_model = xgb.XGBClassifier()

xgb_params = {
    'objective': 'multi:softmax',
    'booster': 'gbtree',
    'eval_metric': 'error',
    'lambda': 2,
    'alpha': 0.02,
    'subsample': 0.8,
    #'colsample_bytree': 1 / F_train.shape[1]**0.5,
    'colsample_bytree': 0.8,
#     'min_child_weight': 4,
    'silent': 1,
    'num_class':3,
    'max_depth':8,
    'min_child_weight':50,
           'eta':0.2,
            'seed':88888
}
num_round=5000
#watchlist=[(dtrain,'train'),(dval,'eval')]

#gbdt = xgb.train(xgb_params, dtrain,num_round,watchlist,early_stopping_rounds=50)
gbdt = xgb.train(xgb_params, dtrain,num_round)
# def ceate_feature_map(features):
#     outfile = open('xgb.fmap', 'w')
#     i = 0
#     for feat in features:
#         outfile.write('{0}\t{1}\tq\n'.format(i, feat))
#         i = i + 1

#     outfile.close()
# ceate_feature_map(list(train.columns))
# print(train.columns)
# xgb.plot_importance(gbdt)
# # gbdt=gbdt
# importance = gbdt.get_fscore(fmap='xgb.fmap')
# importance = sorted(importance.items(), key=operator.itemgetter(1))
# print(importance)
# df = pd.DataFrame(importance, columns=['feature', 'fscore'])
# df['fscore'] = df['fscore'] / df['fscore'].sum()
# #print df
# plt.figure()
# df.plot()
# df.plot(kind='barh', x='feature', y='fscore', legend=False)
# plt.title('XGBoost Feature Importance')
# plt.xlabel('relative importance')
# plt.gcf().savefig('feature_importance_xgb.png')

In [23]:
x_test

,compound,neg,neu,pos
1728,-0.9887,0.131,0.752,0.118
1729,-0.9971,0.136,0.768,0.096
1730,0.9982,0.057,0.853,0.09
1731,0.9843,0.086,0.817,0.097
1732,-0.9977,0.122,0.783,0.095
1733,-0.9996,0.153,0.764,0.083
1734,-0.991,0.125,0.777,0.098
1735,-0.9981,0.113,0.807,0.081
1736,0.9921,0.089,0.812,0.099
1737,0.9995,0.072,0.805,0.124


In [24]:
pred = gbdt.predict(dtest)
# pred = pred.astype(int) 
# size = len(pred)
# i = 0
# while i <size:
#     if pred[i]<0.5:
#         pred[i]= 0
#     else:
#         pred[i]=1
#     i = i+1
print(pred)
y_test
# pred = pred.astype(int) 

[ 0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  1.  1.  0.  0.  0.  0.  1.  1.
  1.  1.  1.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.  0.  1.  1.  1.  1.
  0.  0.  0.  1.  1.  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.  0.  1.  0.
  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  1.  1.  0.  1.  0.  1.
  0.  0.  0.  0.  0.  1.  1.  1.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.
  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  0.  0.  1.  0.  1.  0.  1.  1.
  1.  0.  1.  1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  1.  1.  1.
  0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.  1.  1.  0.  1.
  1.  0.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.
  1.  0.  0.  0.  1.  0.  1.  1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.
  1.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  1.]


1728    1.0
1729    0.0
1730    0.0
1731    0.0
1732    1.0
1733    1.0
1734    1.0
1735    1.0
1736    1.0
1737    0.0
1738    1.0
1739    1.0
1740    1.0
1741    0.0
1742    1.0
1743    1.0
1744    1.0
1745    0.0
1746    1.0
1747    0.0
1748    0.0
1749    1.0
1750    1.0
1751    1.0
1752    1.0
1753    1.0
1754    0.0
1755    1.0
1756    1.0
1757    1.0
       ... 
1895    0.0
1896    0.0
1897    1.0
1898    0.0
1899    0.0
1900    0.0
1901    0.0
1902    0.0
1903    0.0
1904    0.0
1905    1.0
1906    1.0
1907    0.0
1908    1.0
1909    0.0
1910    1.0
1911    1.0
1912    0.0
1913    1.0
1914    1.0
1915    1.0
1916    1.0
1917    0.0
1918    1.0
1919    1.0
1920    0.0
1921    0.0
1922    0.0
1923    1.0
1924    0.0
Name: Adj Close, Length: 197, dtype: float64

In [25]:
# real = pd.read_csv('Ans/test.csv')
# real
# real_1 = real.iloc[:,1]
# real_2 = real.iloc[:,3]
# real_1 = pd.read_csv('Ans/test.csv')
# real_2 = pd.read_csv('Ans/pred.csv')

In [26]:
size = len(pred)
i = 0
count = 0
while i < size:
    if pred[i]==y_test.iloc[i]:
        count = count + 1
    i = i +1
accuracy = count/size *100
print(accuracy)

46.7005076142132


In [22]:
pred

array([ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,
        0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,
        1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
        1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,
        1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,
        1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,
        0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,
        1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,
        1.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1

In [23]:
import pickle

In [24]:
# s = pickle.dumps(gbdt)
# clf2 = pickle.loads(s)
# pred1= clf2.predict(dtest)
# pred1
filename = 'pnb_model_better.pkl'
pickle.dump(gbdt, open(filename, 'wb'),protocol=2)

In [25]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(dtest)
print(result)

[ 0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  1.  1.  0.  0.  0.  0.  1.  1.
  1.  1.  1.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.  0.  1.  1.  1.  1.
  0.  0.  0.  1.  1.  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.  0.  1.  0.
  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  1.  0.  1.  1.  0.  1.  0.  1.
  0.  0.  0.  0.  0.  1.  1.  1.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.
  0.  0.  0.  0.  1.  1.  0.  1.  1.  1.  0.  0.  1.  0.  1.  0.  1.  1.
  1.  0.  1.  1.  1.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  1.  1.  1.
  0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.  1.  1.  0.  1.
  1.  0.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.
  1.  0.  0.  0.  1.  0.  1.  1.  1.  1.  0.  0.  1.  0.  0.  0.  0.  0.
  1.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  1.]


In [26]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pickle
import unicodedata
article = ['']
sid = SentimentIntensityAnalyzer()
# # for date, row in df_stocks.T.iteritems():
# #     try:
# sentence = unicodedata.normalize('NFKD', article)#.encode('ascii','ignore')
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.predict(dtest)
# print(result)
def mohan():
    ss = sid.polarity_scores(article[0])
    answ = {
        'index': 0,
        'compound':ss['compound'],

        'neg':ss['neg'],

        'neu':ss['neu'],

        'pos':ss['pos'],
    }
    lol = []
    lol.append(answ)
    answ = pd.DataFrame(lol)
#     print(answ)
    answ = answ.iloc[:,[0,2,3,4]]
    check = xgb.DMatrix(np.array(answ))
    result = loaded_model.predict(check)
    print(result)
    
mohan()
# answ
# ss['compound']

[ 1.]


In [27]:
from sklearn.cross_validation import KFold
import xgboost as xgb
from sklearn import  metrics
a = np.array([])
kfolds = KFold(df_stocks.shape[0], n_folds=6)
for dev_index, val_index in kfolds:
    dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    dtrain = xgb.DMatrix(dev_X,label = dev_y)
    dtest = xgb.DMatrix(val_X)
    bst = xgb.train( plst,dtrain, num_rounds)
    ypred_bst = bst.predict(dtest,ntree_limit=bst.best_iteration)
    score = metrics.confusion_matrix(val_y, ypred_bst)
    res = (score[0][0]+score[1][1])*1.0/sum(sum(score))
    a = np.append(a,[res])
    print ("Accuracy = %.7f" % (res))
print ("Overall Mean Accuracy = %.7f" % (np.mean(a)))

NameError: name 'train_X' is not defined

In [ ]:
print(Accuracy)